In [4]:
import sys
%pip install weaviate-client

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install getpass 

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass
Note: you may need to restart the kernel to use updated packages.


In [9]:


from getpass import getpass # hide password
import weaviate # to communicate to the Weaviate instance
from weaviate.tools import WCS



In [11]:
my_credentials = weaviate.auth.AuthClientPassword(username="johannschmidt045@gmail.com", password="Ja3g3r31n$")

In [12]:


my_wcs = WCS(my_credentials)



In [13]:
weaviate_url = "https://flabbergoop-1ucb4q7e.weaviate.network"

In [14]:


client = weaviate.Client(weaviate_url, auth_client_secret=my_credentials)



In [15]:
%pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [16]:
import nltk # it is a dependency of newspaper3k
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import newspaper
import uuid
import json
from tqdm import tqdm

def get_articles_from_newspaper(
        news_url: str, 
        max_articles: int=100
    ) -> None:
    """
    Download and save newspaper articles as weaviate schemas.
    Parameters
    ----------
    newspaper_url : str
        Newspaper title.
    """
    
    objects = []
    
    # Build the actual newspaper    
    news_builder = newspaper.build(news_url, memoize_articles=False)
    
    if max_articles > news_builder.size():
        max_articles = news_builder.size()
    pbar = tqdm(total=max_articles)
    pbar.set_description(f"{news_url}")
    i = 0
    while len(objects) < max_articles and i < news_builder.size():
        article = news_builder.articles[i]
        try:
            article.download()
            article.parse()
            article.nlp()

            if (article.title != '' and \
                article.title is not None and \
                article.summary != '' and \
                article.summary is not None and\
                article.authors):

                # create an UUID for the article using its URL
                article_id = uuid.uuid3(uuid.NAMESPACE_DNS, article.url)

                # create the object
                objects.append({
                    'id': str(article_id),
                    'title': article.title,
                    'summary': article.summary,
                    'authors': article.authors
                })
                
                pbar.update(1)

        except:
            # something went wrong with getting the article, ignore it
            pass
        i += 1
    pbar.close()
    return objects

In [18]:


data = []
data += get_articles_from_newspaper('https://www.theguardian.com/international')
data += get_articles_from_newspaper('http://cnn.com')



http://cnn.com: 100%|██████████| 100/100 [00:45<00:00,  2.20it/s]


In [19]:
article_class_schema = {
    # name of the class
    "class": "Article",
    # a description of what this class represents
    "description": "An Article class to store the article summary and its authors",
    # class properties
    "properties": [
        {
            "name": "title",
            "dataType": ["string"],
            "description": "The title of the article", 
        },
        {
            "name": "summary",
            "dataType": ["text"],
            "description": "The summary of the article",
        },
        {
            "name": "hasAuthors",
            "dataType": ["Author"],
            "description": "The authors this article has",
        }
    ]
}

In [20]:
author_class_schema = {
    "class": "Author",
    "description": "An Author class to store the author information",
    "properties": [
        {
            "name": "name",
            "dataType": ["string"],
            "description": "The name of the author", 
        },
        {
            "name": "wroteArticles",
            "dataType": ["Article"],
            "description": "The articles of the author", 
        }
    ]
}

In [21]:
client.schema.create_class(article_class_schema)

UnexpectedStatusCodeException: Add properties to classes! Unexpected status code: 422, with response body: {'error': [{'message': "property 'hasAuthors': invalid dataType: reference property to nonexistent class"}]}

In [ ]:
# helper function
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))



prettify(client.schema.get())

In [ ]:


client.schema.create_class(author_class_schema)
prettify(client.schema.get())



In [ ]:
client.schema.property.create('Article', article_class_schema['properties'][2])

In [ ]:


prettify(client.schema.get())



In [ ]:


schema = client.schema.get() # save schema
client.schema.delete_all() # delete all classes
prettify(client.schema.get())



In [ ]:


client.schema.create(schema)
prettify(client.schema.get())

